## Part 1: Preprocessing

In [340]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [341]:
def preprocess_ohe(df, ohe_column_list):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # List to store DataFrames of encoded columns
    encoded_dfs = []

    # Iterate over each column specified for one-hot encoding
    for column in ohe_column_list:
        # Initialize the OneHotEncoder
        ohe_encoder = OneHotEncoder(handle_unknown='error', sparse_output=False, drop='first') # drop='first'
        
        # Fit and transform the data of the column
        ohe_encoded = ohe_encoder.fit_transform(df_copy[[column]])
        
        # Create a DataFrame from the encoded data with appropriate column names
        ohe_encoded_df = pd.DataFrame(ohe_encoded, columns=ohe_encoder.get_feature_names_out(input_features=[column]))
        
        # Append the newly encoded DataFrame to the list
        encoded_dfs.append(ohe_encoded_df)
        
        # Drop the original column from the copy of the DataFrame
        df_copy.drop(column, axis=1, inplace=True)
    
    # Concatenate all the encoded DataFrames with the modified original DataFrame
    df_copy = pd.concat([df_copy] + encoded_dfs, axis=1)
    
    return df_copy

In [342]:
def preprocess_column_ohe(df, column):
    
    # Initialize the OneHotEncoder
    ohe_encode = OneHotEncoder(handle_unknown='error', sparse_output=False) # drop='first'
    
    # Fit and transform the data of the column
    ohe_encoded = ohe_encode.fit_transform(df[[column]])

    
    return ohe_encoded

In [343]:

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [344]:
print("Columns with NaN values:", attrition_df.columns[attrition_df.isna().any()].tolist())
print ()
print (attrition_df.nunique())
print (attrition_df.dtypes)
print (attrition_df.shape)

Columns with NaN values: []

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64
Age                          int64
Attrition                   object
BusinessTravel              object
Department      

In [345]:
# Create y_df with the Attrition and Department columns
y_headers=['Attrition','Department']
y_df= attrition_df[y_headers]

In [346]:
# Create a list of at least 10 column names to use as X data
# keeping all but y_df values
# List comprehension to filter out columns in y_headers


X_headers = ['Education','Age',
             'DistanceFromHome',
             'JobSatisfaction',
             'OverTime',
             'StockOptionLevel',
             'WorkLifeBalance',
             'YearsAtCompany',
             'YearsSinceLastPromotion',
             'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_headers]

# Show the data types for X_df

X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [347]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_object_columns = X_df.select_dtypes(include=['object']).columns
display (X_object_columns)

Index(['OverTime'], dtype='object')

In [348]:
X_df['OverTime'].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [349]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test  =  train_test_split(X_df,y_df,test_size=.21,random_state=42)

In [350]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train_ohe = preprocess_ohe(X_train,X_object_columns)
X_test_ohe = preprocess_ohe(X_test,X_object_columns)


In [351]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_ohe)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_ohe)
X_test_scaled = scaler.transform(X_test_ohe)


In [352]:
# Create a OneHotEncoder for the Department column

department_col = 'Department'
# Fit the encoder to the training data
# done in function

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train = preprocess_column_ohe(y_df,department_col)
y_department_test  = preprocess_column_ohe(y_df,department_col)
display (y_department_train)
display (y_department_test)

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [353]:
# Create a OneHotEncoder for the Attrition column

attrition_col = 'Attrition'
# Fit the encoder to the training data
# done in function

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train = preprocess_column_ohe(y_df,attrition_col)
y_attrition_test  = preprocess_column_ohe(y_df,attrition_col)
display (y_attrition_train)
display (y_attrition_test)

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [354]:
# Find the number of columns in the X training data

all_columns = X_train.columns.tolist()
y_columns = y_train.columns.tolist()
feature_columns = [col for col in all_columns if col not in y_columns]

# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(200, activation='relu')(input_layer)
shared_layer2 = layers.Dense(100, activation='relu')(shared_layer1)


In [359]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_layer1 = layers.Dense(100, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='Department_output')(department_layer1)


In [360]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_layer1 = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='Attrition_output')(attrition_layer1)


In [362]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'Department_output': 'categorical_crossentropy', 'Attrition_output': 'binary_crossentropy'},
              metrics={'Department_output': 'accuracy', 'Attrition_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 10)]         0           []                               
                                                                                                  
 dense_44 (Dense)               (None, 200)          2200        ['input_layer[0][0]']            
                                                                                                  
 dense_45 (Dense)               (None, 100)          20100       ['dense_44[0][0]']               


                                                                                                  
 dense_48 (Dense)               (None, 100)          10100       ['dense_45[0][0]']               
                                                                                                  
 dense_49 (Dense)               (None, 32)           3232        ['dense_45[0][0]']               
                                                                                                  
 Department_output (Dense)      (None, 3)            303         ['dense_48[0][0]']               
                                                                                                  
 Attrition_output (Dense)       (None, 2)            66          ['dense_49[0][0]']               
                                                                                                  
Total params: 36,001
Trainable params: 36,001
Non-trainable params: 0
_______________________________________

In [364]:
# Train the model
model.fit(
    X_train_scaled,
    {'Department_output': y_department_train, 'Attrition_output': y_attrition_train},
    epochs=500,
    batch_size=100,
    validation_split=0.2
)

Epoch 1/500
12/12 [==============================] - 0s 12ms/step - loss: 1.2080 - Department_output_loss: 0.7559 - Attrition_output_loss: 0.4521 - Department_output_accuracy: 0.6613 - Attrition_output_accuracy: 0.8324 - val_loss: 1.2786 - val_Department_output_loss: 0.8516 - val_Attrition_output_loss: 0.4270 - val_Department_output_accuracy: 0.6290 - val_Attrition_output_accuracy: 0.8481
Epoch 2/500
12/12 [==============================] - 0s 6ms/step - loss: 1.2079 - Department_output_loss: 0.7559 - Attrition_output_loss: 0.4520 - Department_output_accuracy: 0.6613 - Attrition_output_accuracy: 0.8324 - val_loss: 1.2795 - val_Department_output_loss: 0.8524 - val_Attrition_output_loss: 0.4271 - val_Department_output_accuracy: 0.6290 - val_Attrition_output_accuracy: 0.8481
Epoch 3/500
12/12 [==============================] - 0s 5ms/step - loss: 1.2079 - Department_output_loss: 0.7558 - Attrition_output_loss: 0.4521 - Department_output_accuracy: 0.6613 - Attrition_output_accuracy: 0.8324

In [ ]:
# Evaluate the model with the testing data


In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 